In [18]:
%%capture

!pip3 install pandas
!pip3 install numpy

import pandas as pd 
import numpy as np

In [19]:
#Contains movieid tag id and relevance
genome_scores = pd.read_csv('genome-scores.csv')

#contains tagId and tag looks like genres
genome_tags = pd.read_csv('genome-tags.csv')

#contains movie id and corresponding imdbId and tmdbId
links = pd.read_csv('links.csv')

# contains movieId title and genres that the movie falls under
movies = pd.read_csv('movies.csv')

# contains ratings for movieId by users with a timestampe(unsure what this is)
ratings = pd.read_csv('ratings.csv')

#contains UserId associated with a movieId with the tag the user rated with and a time stamp
tags = pd.read_csv('tags.csv')

In [20]:
#drops userId and timestamp not needed
ratings.drop(['userId','timestamp'], axis = 1, inplace = True)
#adds a column for mean and count for the rating 
ratings = ratings.groupby('movieId').agg(['mean','count'])
#renames columns
ratings.columns = ['Average_Rating', 'Total_Rating']

# merges ratings with links csv by movieId
movies = movies.merge(ratings, how = 'inner', on = 'movieId')
movies = movies.merge(links,  how ='inner', on = 'movieId')

In [21]:
#creates a dictionary for title and year 
moviesByYear = []
for x in range(len(movies)):
    moviesByYear.append([
        int(movies.movieId[x]), 
        str(movies.title[x][0:movies.title[x].rfind('(')]),
        str(movies.title[x][movies.title[x].rfind('(')+1:movies.title[x].rfind(')')])
    ])

In [22]:
#replaces current title with new title and year
moviesByYearDf = pd.DataFrame(moviesByYear, columns=['movieId',"title","year"])
movies.drop(columns='title', inplace = True)
movies = moviesByYearDf.merge(movies, how = 'inner', on = 'movieId')

In [23]:
movies.to_csv('DataforMovies.csv')

In [ ]:
#Apparenty nothing needs to be done with this because genres are already added by relevance

#keeping this in case filteration is needed
genome_scores['Genre'] = genome_scores['tagId'].map(genome_tags.set_index('tagId')['tag'])
genome_scores = genome_scores[genome_scores['relevance'] > .75]
genome_scores.reset_index(drop=True, inplace=True)